In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip -q install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 24.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00:00:0100:01


In [3]:
import torch
from transformers import AutoModelForCausalLM,BitsAndBytesConfig

quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
model_4bit=AutoModelForCausalLM.from_pretrained(
    'bigscience/bloom-1b7',
    device_map='cuda',
    quantization_config=quantization_config
)

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

2025-07-24 09:16:57.956634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753348618.140210      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753348618.192612      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

In [5]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained('bigscience/bloom-1b7')
input_data=tokenizer('讲一个笑话:',return_tensors='pt').to('cuda:0')
predict=model_4bit.generate(**input_data,max_new_tokens=100)

indices=predict[0].tolist()
tokenizer.decode(indices)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

'讲一个笑话: 某天,一个老头子,在街上走着,突然,他看见一个小孩,手里拿着一个大西瓜,正津津有味地吃着。 老头子问:"你吃的是什么呀？" 小孩说:"西瓜。" 老头子说:"你吃西瓜,我吃西瓜,你吃西瓜,我吃西瓜,你吃西瓜,我吃西瓜,你吃西瓜,我吃西瓜,你吃西瓜,我吃西瓜,你吃西瓜,'

In [6]:
input_data=tokenizer('请解释什么是大语言模型:',return_tensors='pt').to('cuda:0')
predict=model_4bit.generate(**input_data,max_new_tokens=100)

indices=predict[0].tolist()
tokenizer.decode(indices)

'请解释什么是大语言模型:大语言模型是针对大数据量、高并发、高性能、高安全、高可靠、高可扩展性等特点,采用分布式、高并发、高性能、高安全、高可靠、高可扩展性等特点,采用分布式、高并发、高性能、高安全、高可靠、高可扩展性等特点,采用分布式、高并发、高性能、高安全、高可靠、高可扩展性等特点,采用分布式'